In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

import glob
import imageio.v3 as iio
import numpy as np
import os
import pandas as pd
import random

In [ ]:
SUBJ_LIST = ["subj01", "subj02", "subj03", "subj04", "subj05", "subj06", "subj07", "subj08"]

In [ ]:
ROOT_FILE_PATH = "/content/drive/My Drive/Colab Notebooks/2023-Machine-Learning-Dataset"

def ReadNpyFile(dir_path, name):
  dic = {}
  i = 1
  dic[0] = name
  for path in glob.glob(dir_path + "/*.npy"):
    dic[i] = {}
    dic[i][0] = path.split("/")[-1]
    dic[i][1] = np.load(path, allow_pickle = True)
    i += 1
  return dic

def ReadPngFile(dir_path, feature, table):
  dic = {}
  i = 0
  dic[5000] = feature
  for path in glob.glob(dir_path + "/*.png"):
    dic[i] = {}
    dic[i][0] = path.split("/")[-1]
    dic[i][1] = iio.imread(path)
    dic[i][2] = {}
    dic[i][3] = table[i]
    for j in range(425):
      dic[i][2][j] = {}
      for k in range(425):
         dic[i][1][j][k] = 0.299 * dic[i][1][j][k][0] + 0.587 * dic[i][1][j][k][1] + 0.114 * dic[i][1][j][k][2]
    i += 1
  return dic

In [ ]:
#readme
"""
Subject 1-8:

image[0-4999] --> [0] --> name
                  [1] --> RGB([0-455][0-455]pixel)(value:0-255)
                  [2] --> gray scale
                  [3] --> tage(133 labels)(0 or 1)
                  
fmri[0-4999]  --> [0] --> left
                  [1] --> right

mask[0-31]    --> [0] --> name(type of mask)
                  [1] --> np_array
"""

In [ ]:
class ReadObject:
  
  subject = None
  images = None
  fmri = []
  fmri_ex = None
  masks = None
  image_data = None
  table = None

  def get_info(self, subject):
    
    self.table = {}
    image_data = pd.read_csv(os.path.join(ROOT_FILE_PATH, "image_infos", f"{subject}_infos_train.csv"))

    for i in range(5000):
      
      self.table[i] = {}
      
      j = 2
      for feature in image_data.columns:
        
        if j > 0:
          j -= 1
          continue
            
        self.table[i] = np.array(int(image_data.loc[i, feature]))
        eat = [feature]
        outp = np.array(eat)
            
    return outp

  ## Mode = "train" | "test"
  def load(self, subject, temp, mode = "train"):
    
    if subject != self.subject:
      self.subject = subject
      self.masks = ReadNpyFile(os.path.join(ROOT_FILE_PATH, subject , "roi_masks"), "roi_masks")

    if mode == "train":
      self.images = ReadPngFile(os.path.join(ROOT_FILE_PATH, subject, "training_split", "training_images"), temp, self.table)
      self.fmri_ex = ReadNpyFile(os.path.join(ROOT_FILE_PATH, subject, "training_split", "training_fmri"))

    elif mode == "test":
      self.images = None
      self.fmri_ex = ReadNpyFile(os.path.join(ROOT_FILE_PATH, subject, "test_split", "test_fmri"))

  ## batch_number = probability of
  def split_data(self, batch_number = 1):
    training_set = []
    validation_set = []
    for i in range(5000):
        eat = random.random()
        if eat < batch_number:
            training_set.append(self.image[i])
        else:
            validation_set.append(self.image[i])
    training_set = np.array(training_set)
    validation_set =np.array(validation_set)
    return training_set, validation_set

  def read_fmri(self):
    for i in range(5000):
      self.fmri[i] = []
      self.fmri[i][0] = self.fmri_ex[0][1][i]  #left
      self.fmri[i][1] = self.fmri_ex[1][1][i]  #right

In [ ]:
combined_data = ReadObject()

subject = SUBJ_LIST[0]

temp = combined_data.get_info(subject)
combined_data.load(subject, temp)

combined_data.read_fmri()

data_train, data_valid = combined_data.splitted_data(0.9)
# np.save('/content/drive/My Drive/Colab Notebooks/data_valid.npy', data_valid)
# np.save('/content/drive/My Drive/Colab Notebooks/data_train.npy', data_train)